# End to End Pure Streaming Data-Pipeline for Apartment Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read apartment state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "apartment_streamingDF" --> "results_df" --> "apartment_data"

In [0]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [0]:
# Schema for Apartment JSON
apartment_schema = StructType([
            StructField("Building_id", IntegerType(), True),
            StructField("Apartment_number", IntegerType(), True),
            StructField("Type", StringType(), True),
            StructField("Rent_fee", StringType(), True),
            StructField("Building_name", StringType(), True),
            StructField("Appt_details", StringType(), True),
            StructField("event_time", TimestampType(), True),
            StructField("fetch_time", StringType(), True)])

apartment_udf_schema = StructType([
            StructField("Apartment_number", IntegerType(), True),
            StructField("Type", StringType(), True),
            StructField("Rent_fee", StringType(), True),
            StructField("Building_name", StringType(), True),
            StructField("Appt_details", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get apartment csv files as a streaming "apartment_streamingDF" and process it on the fly and get transformed stream "apartment_df"
###### Objective: (incoming csv files) --> "apartment_streamingDF" --> "apartment_df"

In [0]:
# Get Apartment Steaming DataFrame from csv files

# streaming starts here by reading the input files 
apartment_Path = "/FileStore/apartment/apartment/inprogress/"
apartment_streamingDF = (
  spark
    .readStream
    .schema(apartment_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(apartment_Path)
)
# Clear invalid rows
apartment_df = apartment_streamingDF.select("*").where("Building_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(apartment_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Type', 'Rent_fee', 'Building_name', 'Appt_details'])

Out[ 288 ]: <optimus.df_transformer.DataFrameTransformer at 0x7f5c266f99e8>

##### This function parses the corresponding columns into a single column

In [0]:
def my_fun(Apartment_number, Type, Rent_fee, Building_name, Appt_details, event_time):
  return zip(Apartment_number, Type, Rent_fee, Building_name, Appt_details, event_time)

udf_Fun = udf(my_fun, ArrayType(apartment_udf_schema))

In [0]:
intermediate_df = ( apartment_df.withWatermark("event_time", "10 seconds")
            .groupBy("Building_id")
            .agg(F.collect_list("Apartment_number").alias("Apartment_number"),
                 F.collect_list("Type").alias("Type"),
                 F.collect_list("Rent_fee").alias("Rent_fee"),
                 F.collect_list("Building_name").alias("Building_name"),
                 F.collect_list("Appt_details").alias("Appt_details"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Building_id", 
                    F.explode(udf_Fun(F.column("Apartment_number"), 
                                      F.column("Type"), 
                                      F.column("Rent_fee"), 
                                      F.column("Building_name"), 
                                      F.column("Appt_details"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [0]:
results_df = (intermediate_df
              .select("Building_id", 
                      "data.Apartment_number", 
                      "data.Type", 
                      "data.Rent_fee", 
                      "data.Building_name", 
                      "data.Appt_details", 
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Building_id")

##### Display final result
###### This result shows the latest state of all the unique Building_id

In [0]:
display(results_df)

Building_id,Apartment_number,Type,Rent_fee,Building_name,Appt_details,event_time,latest_event_time
1,1,Twin Room,$0.21,"Hirthe, Feil and Schneider","Integer tincidunt ante vel ipsum. Praesent blandit lacinia erat. Vestibulum sed magna at nunc commodo placerat. Praesent blandit. Nam nulla. Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula. Suspendisse ornare consequat lectus. In est risus, auctor sed, tristique in, tempus sit amet, sem. Fusce consequat. Nulla nisl. Nunc nisl. Duis bibendum, felis sed interdum venenatis, turpis enim blandit mi, in porttitor pede justo eu massa. Donec dapibus. Duis at velit eu est congue elementum. In hac habitasse platea dictumst. Morbi vestibulum, velit id pretium iaculis, diam erat fermentum justo, nec condimentum neque sapien placerat ante. Nulla justo. Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis. Sed ante. Vivamus tortor. Duis mattis egestas metus. Aenean fermentum. Donec ut mauris eget massa tempor convallis. Nulla neque libero, convallis eget, eleifend luctus, ultricies eu, nibh.",2018-02-10T14:11:13.000+0000,2018-02-10T14:11:13.000+0000
2,2,One Bedroom Flat,$6.18,Wiegand-Upton,"Nullam porttitor lacus at turpis. Donec posuere metus vitae ipsum. Aliquam non mauris. Morbi non lectus. Aliquam sit amet diam in magna bibendum imperdiet. Nullam orci pede, venenatis non, sodales sed, tincidunt eu, felis. Fusce posuere felis sed lacus. Morbi sem mauris, laoreet ut, rhoncus aliquet, pulvinar sed, nisl. Nunc rhoncus dui vel sem. Sed sagittis. Nam congue, risus semper porta volutpat, quam pede lobortis ligula, sit amet eleifend pede libero quis orci. Nullam molestie nibh in lectus. Pellentesque at nulla. Suspendisse potenti. Cras in purus eu magna vulputate luctus. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Vivamus vestibulum sagittis sapien. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Etiam vel augue. Vestibulum rutrum rutrum neque. Aenean auctor gravida sem. Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio.",2017-01-21T11:29:43.000+0000,2017-01-21T11:29:43.000+0000
3,3,One Bedroom Flat,$2.81,Feest LLC,"Curabitur in libero ut massa volutpat convallis. Morbi odio odio, elementum eu, interdum eu, tincidunt in, leo. Maecenas pulvinar lobortis est. Phasellus sit amet erat. Nulla tempus. Vivamus in felis eu sapien cursus vestibulum. Proin eu mi. Nulla ac enim. In tempor, turpis nec euismod scelerisque, quam turpis adipiscing lorem, vitae mattis nibh ligula nec sem.",2018-05-30T19:03:52.000+0000,2018-05-30T19:03:52.000+0000
4,4,Single Bedroom,$9.03,"Rice, Kihn and Dach","Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Vivamus vestibulum sagittis sapien. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Etiam vel augue. Vestibulum rutrum rutrum neque. Aenean auctor gravida sem. Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl.",2018-05-06T20:04:12.000+0000,2018-05-06T20:04:12.000+0000
5,5,Single Studio Flat,$0.21,Romaguera LLC,"Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl. Aenean l

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [0]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/_checkpoints/streaming-agg")
 .start("/delta/apartment/apartment_data"))

Out[ 291 ]: <pyspark.sql.streaming.StreamingQuery at 0x7f5c2670b240>

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe wil always be Up-To-Date

In [0]:
apartment_data = spark.read.format("delta").load("/delta/apartment/apartment_data").orderBy("Building_id")

In [0]:
display(apartment_data)

Building_id,Apartment_number,Type,Rent_fee,Building_name,Appt_details,event_time,latest_event_time
1,1,Twin Room,$0.21,"Hirthe, Feil and Schneider","Integer tincidunt ante vel ipsum. Praesent blandit lacinia erat. Vestibulum sed magna at nunc commodo placerat. Praesent blandit. Nam nulla. Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula. Suspendisse ornare consequat lectus. In est risus, auctor sed, tristique in, tempus sit amet, sem. Fusce consequat. Nulla nisl. Nunc nisl. Duis bibendum, felis sed interdum venenatis, turpis enim blandit mi, in porttitor pede justo eu massa. Donec dapibus. Duis at velit eu est congue elementum. In hac habitasse platea dictumst. Morbi vestibulum, velit id pretium iaculis, diam erat fermentum justo, nec condimentum neque sapien placerat ante. Nulla justo. Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis. Sed ante. Vivamus tortor. Duis mattis egestas metus. Aenean fermentum. Donec ut mauris eget massa tempor convallis. Nulla neque libero, convallis eget, eleifend luctus, ultricies eu, nibh.",2018-02-10T14:11:13.000+0000,2018-02-10T14:11:13.000+0000
2,2,One Bedroom Flat,$6.18,Wiegand-Upton,"Nullam porttitor lacus at turpis. Donec posuere metus vitae ipsum. Aliquam non mauris. Morbi non lectus. Aliquam sit amet diam in magna bibendum imperdiet. Nullam orci pede, venenatis non, sodales sed, tincidunt eu, felis. Fusce posuere felis sed lacus. Morbi sem mauris, laoreet ut, rhoncus aliquet, pulvinar sed, nisl. Nunc rhoncus dui vel sem. Sed sagittis. Nam congue, risus semper porta volutpat, quam pede lobortis ligula, sit amet eleifend pede libero quis orci. Nullam molestie nibh in lectus. Pellentesque at nulla. Suspendisse potenti. Cras in purus eu magna vulputate luctus. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Vivamus vestibulum sagittis sapien. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Etiam vel augue. Vestibulum rutrum rutrum neque. Aenean auctor gravida sem. Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio.",2017-01-21T11:29:43.000+0000,2017-01-21T11:29:43.000+0000
3,3,One Bedroom Flat,$2.81,Feest LLC,"Curabitur in libero ut massa volutpat convallis. Morbi odio odio, elementum eu, interdum eu, tincidunt in, leo. Maecenas pulvinar lobortis est. Phasellus sit amet erat. Nulla tempus. Vivamus in felis eu sapien cursus vestibulum. Proin eu mi. Nulla ac enim. In tempor, turpis nec euismod scelerisque, quam turpis adipiscing lorem, vitae mattis nibh ligula nec sem.",2018-05-30T19:03:52.000+0000,2018-05-30T19:03:52.000+0000
4,4,Single Bedroom,$9.03,"Rice, Kihn and Dach","Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Vivamus vestibulum sagittis sapien. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Etiam vel augue. Vestibulum rutrum rutrum neque. Aenean auctor gravida sem. Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl.",2018-05-06T20:04:12.000+0000,2018-05-06T20:04:12.000+0000
5,5,Single Studio Flat,$0.21,Romaguera LLC,"Praesent id massa id nisl venenatis lacinia. Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl. Aenean l

### Do Some Live Streaming Graphs

In [0]:
apartment_data_stream = spark.readStream.format("delta").load("/delta/apartment/apartment_data")

In [0]:
display(apartment_data_stream.groupBy("Type").count())

Type,count
One Bedroom Flat,31
Single Bedroom with Washbasin,2
Single Bedroom,2
Single Studio Flat,6
Twin Room,3
Single Ensuite,5
